In [1]:
import numpy as np
import cv2
img = cv2.imread('./col.jpg')


In [2]:
# 图像大小
height, width = img.shape[:2]
print(height,width)
# 生成网格点
x, y = np.meshgrid(np.arange(width), np.arange(height))
pixel_coords = np.stack([x, y, np.ones_like(x)], axis=-1)
print(pixel_coords.shape)


# print(pixel_coords[:,-1,:])

559 825
(559, 825, 3)


In [3]:
# 相机内参
f = 500
K = np.array([[f, 0, width/2], [0, f, height/2], [0, 0, 1]])
# cam_ray = np.matmul(np.linalg.inv(K), pixel_coords)

K_inv = np.linalg.inv(K)
# np.matmul(K_inv,pixel_coords)
# 将 K_inv 广播到 (3, 3, height, width)
# K_inv_bcast = np.tile(K_inv, (1, 1, pixel_coords.shape[1],pixel_coords.shape[0]))

# print(K_inv_bcast.shape)
# 计算每个像素的光线经过水面折射后的方向"
# print(np.array([0,0,0]).shape)
# cam_ray = np.matmul(K_inv_bcast, pixel_coords)
# cam_ray = cam_ray / np.linalg.norm(cam_ray, axis=-1, keepdims=True)
cam_ray = np.matmul(K_inv, pixel_coords.reshape(-1, 3).T).T.reshape(pixel_coords.shape)
print(cam_ray.shape)

# 相机坐标系： 对应像素的单位方向向量
cam_ray_norm = cam_ray / np.linalg.norm(cam_ray, axis=-1, keepdims=True)
print(cam_ray.shape)

(559, 825, 3)
(559, 825, 3)


In [4]:
# 计算折射角度
air_refractive_index = 1.0
water_refractive_index = 1.33
sin_theta1 = cam_ray_norm[..., -1]
print(sin_theta1.shape)

sin_theta2 = air_refractive_index / water_refractive_index * sin_theta1

# 水下部分光线的方向向量，即折射部分
# refract_ray = (air_refractive_index / water_refractive_index) * cam_ray[..., :-1] + sin_theta2[..., None] * np.array([0, 0, 1])
refract_ray = air_refractive_index / water_refractive_index * cam_ray
print(refract_ray.shape)




(559, 825)
(559, 825, 3)


In [5]:
refract_coords = np.matmul(K,refract_ray.reshape(-1,3).T).T.reshape(pixel_coords.shape)


refract_coords = np.round(refract_coords).astype(int)
print(refract_coords.shape)
# # refract_coords = np.clip(refract_coords, 0, np.array([height-1, width-1]))
# refracted_img = img[refract_coords[..., 1], refract_coords[..., 0]]

# new_r = np.round(refract_coords).astype(int)

new_r = np.clip(refract_coords, 0, np.array([width-1, height-1,1]))


refracted_img = img[refract_coords[..., 1], refract_coords[..., 0]]



(559, 825, 3)


In [6]:
# 显示图片
cv2.imwrite('./tmp.jpg',refracted_img)

True

In [7]:

# # 计算每个像素的折射点
# t = x
# refract_coords = pixel_coords[..., :-1] + t[..., None] * refract_ray[..., :-1]

# 双线性插值采样
refract_coords = np.round(refract_coords).astype(int)
# refract_coords = np.clip(refract_coords, 0, np.array([width-1, height-1]))
# refracted_img = img[refract_coords[..., 1], refract_coords[..., 0]]

In [ ]:
def refract_correct(img, water_refractive_index, f):
    # 图像大小
    height, width = img.shape[:2]

    # 生成网格点
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    pixel_coords = np.stack([x, y, np.ones_like(x)], axis=-1)

    # 相机内参
    K = np.array([[f, 0, width/2], [0, f, height/2], [0, 0, 1]])

    # 计算每个像素的光线经过水面折射后的方向
    cam_ray = np.linalg.inv(K) @ pixel_coords
    cam_ray = cam_ray / np.linalg.norm(cam_ray, axis=-1, keepdims=True)
    
    # 计算折射角度
    air_refractive_index = 1.0
    cos_theta1 = -cam_ray[..., -1]
    cos_theta2 = np.sqrt(1 - (air_refractive_index / water_refractive_index)**2 * (1 - cos_theta1**2))
    
    # 水下部分光线的方向向量，即折射部分
    refract_ray = (air_refractive_index / water_refractive_index) * cam_ray[..., :-1] + cos_theta2[..., None] * np.array([0, 0, -1])
    
    # 计算每个像素的折射点
    t = -cam_ray[..., -1] / refract_ray[..., -1]
    refract_coords = pixel_coords[..., :-1] + t[..., None] * refract_ray[..., :-1]

    # 双线性插值采样
    refract_coords = np.round(refract_coords).astype(int)
    refract_coords = np.clip(refract_coords, 0, np.array([width-1, height-1]))
    refracted_img = img[refract_coords[..., 1], refract_coords[..., 0]]

    return refracted_img
